In [81]:
import pandas as pd
import numpy as np

In [82]:
# initialize state list
states = [
    'ca',
#     'co',
    'dc',
#     'ks',
    'ga',
    'la',
    'ma',
    'mn',
    'mo',
    'nj',
    'pa',
#     'tn',
    'tx'
]

# states=['nj']

finalized_path = './data/finalized/'

In [83]:
def cleanStrings(row, column):
    try:
        return str(int(float(row[column])))
    except:
        return row[column]

In [84]:
# import in_city & charter from directory
df_directory = pd.read_csv(finalized_path + 'directory_data_combined.csv',dtype={'district_id':str,'school_id':str,'district':str,'school':str,'in_city':str,'charter':str,'state':str})
df_directory['state'] = df_directory['state'].str.lower()
df_directory['flag'] = 1
df_directory = df_directory[['district_id', 'school_id', 'in_city', 'charter', 'state', 'flag']]
# apply cleanStrings function to district_id and school_id
df_directory['district_id'] = df_directory.apply(lambda x: cleanStrings(x, 'district_id') ,axis=1)
df_directory['school_id'] = df_directory.apply(lambda x: cleanStrings(x, 'school_id') ,axis=1)

print(df_directory.shape)
print(df_directory.head(5))

(28840, 6)
  district_id school_id in_city charter state  flag
0         NaN       NaN     NaN     NaN    la     1
1           1         1     NaN     NaN    ma     1
2           1         1     NaN     NaN    mn     1
3           1        15     NaN     NaN    ma     1
4           1         2     NaN     NaN    mn     1


In [85]:
# import ed champion
df_champ = pd.read_csv(finalized_path + 'ed_champions_data.csv', dtype={'edchamp_id':str,'edchamp_name':str,'city':str,'state':str,'latitude':np.float64,'longitude':np.float64,'first_grant_start':str,'launch_year_spring':str})
df_champ['state'] = df_champ['state'].str.lower()

# remove The Opportunity Trust from ed champ file for now, per Steve
df_champ = df_champ[~df_champ.edchamp_name.eq('The Opportunity Trust')]

print(df_champ.shape)
print(df_champ)

(10, 8)
   edchamp_id                      edchamp_name          city state  \
0           1               Boston Schools Fund        Boston    ma   
1           2           City Education Partners   San Antonio    tx   
2           3                      EdForward DC    Washington    dc   
3           4                        Educate 78       Oakland    ca   
4           5  New Jersey Children's Foundation        Newark    nj   
5           6       New Schools for Baton Rouge   Baton Rouge    la   
6           7  Philadelphia Schools Partnership  Philadelphia    pa   
7           8                        RedefineED       Atlanta    ga   
8           9                    SchoolSmart KC   Kansas City    mo   
10         11                  Great MN Schools   Minneapolis    mn   

     latitude   longitude first_grant_start launch_year_spring  
0   42.370567  -71.026964           9/24/15               2016  
1   29.468413  -98.528889            9/1/15               2016  
2   38.911936  

In [86]:
# import grant data
df_grant = pd.read_csv(finalized_path + 'grants_clean_unifiedForAnalysis.csv',dtype={'grantmerge': str,'state':str,'educationchampion':str,'grantid':str,'grantscope':str,'district_id':str,'stateleaname':str,'previousstateleaid':str,'schoolgranteeid':str,'school_id':str,'stateschoolname':str,'previousstateschoolid':str,'granteeisthesameasgrantben':str,'granteenameifdifferentfrom':str,'leagrantstrategy':str,'strategyenrollmentmax':str,'strategyqualityimprovement':str,'strategyexpansion':str,'strategynewstart':str,'endofgrantleaenrollmentgoal':str,'granttype':str,'supportorganizationsinvolved':str,'levelofedchampinvolvement':str,'duration':str,'startdate':str,'enddate':str,'totalamountcommitted':np.float64,'totalamountdisbursed':np.float64,'otherfundscontributed':str,'notes':str,'schooltype':str,'schoolgrantstrategy':str,'endofgrantenrollmentgoal':np.float64,'wasanyportionofthetotalgra':str,'portionedamountcommitted':np.float64,'portionedamountdisbursed':np.float64,'no_id_status':str}, parse_dates=['startdate', 'enddate'])

# apply cleanStrings function to district_id and school_id
df_grant['district_id'] = df_grant.apply(lambda x: cleanStrings(x, 'district_id') ,axis=1)
df_grant['school_id'] = df_grant.apply(lambda x: cleanStrings(x, 'school_id') ,axis=1)
df_grant['educationchampion'] = ['RedefinEd' if x == 'RedefineEd' else x for x in df_grant['educationchampion']]

# find start date school year
df_grant['year'] = pd.DatetimeIndex(df_grant['startdate']).year
df_grant['month'] = pd.DatetimeIndex(df_grant['startdate']).month

def calcSY(row):
    try:
        if row['month'] >= 6:
            return row['year'] + 1
        else:
            return row['year']
    except:
        return np.nan

df_grant['start_sy'] = df_grant.apply(lambda x: calcSY(x), axis = 1)
df_grant = df_grant.dropna(subset=['start_sy'])
df_grant['start_sy'] = df_grant['start_sy'].astype('int64')

# drop records with start date school year > 2020
df_grant = df_grant[df_grant['start_sy'] <= 2020]

print(df_grant.shape)
print(df_grant['educationchampion'].drop_duplicates())
print(df_grant.head(5))

# join edchamp to grants dataset
print(df_grant.shape)
df_grant = df_grant.merge(df_champ, on=['state'])
print(df_grant.shape)

(1349, 40)
0                             Educate 78
933                         EdForward DC
977                            RedefinEd
992          New Schools for Baton Rouge
1015                 Boston Schools Fund
1066                    Great MN Schools
1283                      SchoolSmart KC
1298    New Jersey Children's Foundation
1302    Philadelphia Schools Partnership
1370             City Education Partners
Name: educationchampion, dtype: object
    grantmerge state educationchampion grantid    grantscope district_id  \
0  matched (3)    ca        Educate 78   11101  School-based       61259   
1  matched (3)    ca        Educate 78   11201  School-based       61259   
2  matched (3)    ca        Educate 78   11301  School-based       61259   
3  matched (3)    ca        Educate 78   11301  School-based       61259   
4  matched (3)    ca        Educate 78   11301  School-based       61259   

                        stateleaname previousstateleaid schoolgranteeid  \
0  Santa

In [87]:
# drop null start_sy
temp = df_grant.dropna(subset=['start_sy'])

# create earliest grant year by state
df_grantyear = temp[['state', 'start_sy']].drop_duplicates()
df_grantyear = df_grantyear.groupby(['state'], as_index=False)['start_sy'].min()
print(df_grantyear)

# create earliest grant year by district
df_grantyear_d = temp[temp.grantscope.eq('LEA-wide')][['state', 'district_id', 'start_sy']].drop_duplicates()
df_grantyear_d = df_grantyear_d.groupby(['state', 'district_id'], as_index=False)['start_sy'].min()
df_grantyear_d = df_grantyear_d.rename(columns={'start_sy': 'start_sy_d'})
print(df_grantyear_d.head(5))

# create earliest grant year by school
df_grantyear_s = temp[temp.grantscope.eq('School-based')][['state', 'district_id', 'school_id', 'start_sy']].drop_duplicates()
df_grantyear_s = df_grantyear_s.groupby(['state', 'district_id', 'school_id'], as_index=False)['start_sy'].min()
df_grantyear_s = df_grantyear_s.rename(columns={'start_sy': 'start_sy_s'})
print(df_grantyear_s.head(5))

  state  start_sy
0    ca      2014
1    dc      2017
2    ga      2018
3    la      2016
4    ma      2016
5    mn      2015
6    mo      2017
7    nj      2020
8    pa      2012
9    tx      2016
  state district_id  start_sy_d
0    ca       61259        2018
1    ga     7830617        2018
2    ga     7830628        2019
3    ma         412        2018
4    ma         416        2018
  state district_id school_id  start_sy_s
0    ca       10017    112607        2016
1    ca       10017    123968        2016
2    ca       10017    124172        2016
3    ca       10017    125567        2016
4    ca       10017    131581        2016


In [88]:
# create grantmaker details

# grantee_district_flag
grants_district = df_grant[df_grant.grantscope.eq('LEA-wide')][['state', 'district_id']].drop_duplicates()
grants_district['grant_flag_district'] = 1
grants_district = grants_district.dropna(subset=['district_id'])
print(grants_district.head())

# grantee_school_flag
grants_school = df_grant[df_grant.grantscope.eq('School-based')][['state', 'district_id', 'school_id']].drop_duplicates()
grants_school['grant_flag_school'] = 1
grants_school = grants_school.dropna(subset=['district_id', 'school_id'])
print(grants_school.head())

     state district_id  grant_flag_district
869     ca       61259                    1
983     ga     7830617                    1
986     ga     7830628                    1
1015    ma         469                    1
1018    ma         444                    1
  state district_id school_id  grant_flag_school
0    ca       61259   6002166                  1
2    ca       61259   6111660                  1
3    ca       61259    129635                  1
4    ca       61259    114868                  1
5    ca       10017    112607                  1


In [89]:
# create grant strategy fields
df_grantdetails = df_grant[['state', 'district_id', 'school_id', 'grantscope', 'strategyenrollmentmax', 'strategyexpansion', 'strategynewstart', 'strategyqualityimprovement', ]]

df_grantdetails['enrollment_flag'] = [1 if x == 'Y' else 0 for x in df_grantdetails['strategyenrollmentmax']]
df_grantdetails['expansion_flag'] = [1 if x == 'Y' else 0 for x in df_grantdetails['strategyexpansion']]
df_grantdetails['newstart_flag'] = [1 if x == 'Y' else 0 for x in df_grantdetails['strategynewstart']]
df_grantdetails['qi_flag'] = [1 if x == 'Y' else 0 for x in df_grantdetails['strategyqualityimprovement']]
df_grantdetails['growth'] = df_grantdetails['enrollment_flag'] + df_grantdetails['expansion_flag']

df_grantdetails_d = df_grantdetails[df_grantdetails.grantscope.eq('LEA-wide')]
df_grantdetails_s = df_grantdetails[df_grantdetails.grantscope.eq('School-based')]

df_grantdetails_d = df_grantdetails_d.groupby(['state', 'district_id'], as_index=False).agg({'growth': 'sum','newstart_flag':'sum', 'qi_flag':'sum'})
df_grantdetails_s = df_grantdetails_s.groupby(['state', 'district_id', 'school_id'], as_index=False).agg({'growth': 'sum','newstart_flag':'sum', 'qi_flag':'sum'})

df_grantdetails_d = df_grantdetails_d.rename(columns = {
    'growth': 'grants_district_growth',
    'newstart_flag': 'grants_district_newstart',
    'qi_flag': 'grants_district_qa'
})

df_grantdetails_s = df_grantdetails_s.rename(columns = {
    'growth': 'grants_school_growth',
    'newstart_flag': 'grants_school_newstart',
    'qi_flag': 'grants_school_qa'
})

print(df_grantdetails_d.head())
print(df_grantdetails_s.head())

  state district_id  grants_district_growth  grants_district_newstart  \
0    ca       61259                       0                         0   
1    ga     7830617                       0                         1   
2    ga     7830628                       0                         1   
3    ma         412                       0                         0   
4    ma         416                       1                         0   

   grants_district_qa  
0                   8  
1                   0  
2                   0  
3                   1  
4                   0  
  state district_id school_id  grants_school_growth  grants_school_newstart  \
0    ca       10017    112607                    21                       2   
1    ca       10017    123968                     9                       0   
2    ca       10017    124172                    12                       1   
3    ca       10017    125567                     9                       1   
4    ca       10017   

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

In [90]:
# import grade map
df_grademap = pd.read_csv(finalized_path + 'grade_lookup.csv',dtype={'state':str,'grade':str,'grade_new':str, 'file':str})

print(df_grademap.shape)
print(df_grademap.head(5))

(301, 4)
  state grade grade_new        file
0    ca     1         1  enrollment
1    ca     2         2  enrollment
2    ca     9    HS/EOC  enrollment
3    ca    10    HS/EOC  enrollment
4    ca    12    HS/EOC  enrollment


In [91]:
# import group map
df_groupmap = pd.read_csv(finalized_path + 'group_lookup.csv',dtype={'state':str,'group_state':str,'group_new':str, 'file':str})

print(df_groupmap.shape)
print(df_groupmap.head(5))

(167, 4)
  state       group_state     group_new        file
0    ca  African American         Black  enrollment
1    ca          Hispanic      Hispanic  enrollment
2    ca             White         White  enrollment
3    ca           frplk12    Low-Income  enrollment
4    ca          totalk12  All Students  enrollment


In [92]:
# import subject map
df_subjectmap = pd.read_csv(finalized_path + 'subject_lookup.csv',dtype={'state':str,'subject':str,'subject_new':str, 'file':str})

print(df_subjectmap.shape)
print(df_subjectmap.head(5))

(84, 4)
  state subject subject_new         file
0    ca     ELA         ELA  proficiency
1    ca    Math        Math  proficiency
2    co     ELA         ELA  proficiency
3    co    Math        Math  proficiency
4    dc     ELA         ELA  proficiency


In [93]:
# import enrollment files
df_e = pd.DataFrame()

for state in states:
    try:
        temp = pd.read_csv(finalized_path + state + '_enrollment.csv',dtype={'year': str,'district_id': str,'district': str,'school_id': str,'school': str,'grade': str,'group_state': str,'num': np.int64})
    except: 
        print('No enrollment file for ' + state)
        continue
    temp['state'] = state
    print('Importing ' + state)
    # apply cleanStrings function to district_id and school_id
    temp['district_id'] = temp.apply(lambda x: cleanStrings(x, 'district_id') ,axis=1)
    temp['school_id'] = temp.apply(lambda x: cleanStrings(x, 'school_id') ,axis=1)
    
    print('Pre city map')
    print(temp.shape)
    # join city file
    if state == 'tx':
        temp_dir = df_directory[['state', 'district_id', 'in_city', 'charter', 'flag']].drop_duplicates()
        temp = temp.merge(temp_dir, on=['state', 'district_id'], how = 'left')
        del temp_dir
    else:
        temp = temp.merge(df_directory, on=['state', 'district_id', 'school_id'], how = 'left')
        temp['charter'] = temp.apply(lambda x: '1' if x['state'] == 'dc' and str(x['district_id']).strip() != '1' else x['charter'], axis = 1)
        temp['in_city'] = temp.apply(lambda x: '1' if x['state'] == 'dc' else x['in_city'], axis = 1)
    del temp['flag']
    print('Post city map')
    print(temp.shape)
    temp['charter'] = temp['charter'].fillna('0')
    temp['in_city'] = temp['in_city'].fillna('0')
    
    df_e = df_e.append(temp, ignore_index = True, sort = True)

print('After all appends')
print(df_e.shape)
# join groupmap and grademap
df_e = df_e.merge(df_grademap, on=['state','grade'], how='left')
df_e = df_e.merge(df_groupmap, on=['state','group_state'], how='left')
df_e[df_e.grade_new.isnull()][['state', 'grade']].drop_duplicates().to_csv('unmatched_grade_enrollment.csv')
df_e[df_e.group_new.isnull()][['state', 'group_state']].drop_duplicates().to_csv('unmatched_group_enrollment.csv')
df_e = df_e.groupby(['year', 'state', 'district_id', 'district', 'school_id', 'school', 'grade_new', 'group_new', 'in_city', 'charter'], as_index=False)['num'].sum()
df_e = df_e.rename(columns={'grade_new': 'grade', 'group_new': 'group'})
print('After grade/group map')
print(df_e.shape)
# join grants data to enrollment data
# df_e = df_e.merge(grants_district, on=['state', 'district_id'], suffixes=('', '_grant_district'), how='left')
# df_e = df_e.merge(grants_school, on=['state', 'district_id', 'school_id'], suffixes=('', '_grant_school'), how='left')
# print('After grant join')
# print(df_e.shape)

# join grant year
df_e = df_e.merge(df_grantyear, on=['state'], how='left')
print('After grant year join')
print(df_e.shape)

# join grant year district
df_e = df_e.merge(df_grantyear_d, on=['state', 'district_id'], how='left')
print('After grant year district join')
print(df_e.shape)

# join grant year school
df_e = df_e.merge(df_grantyear_s, on=['state', 'district_id', 'school_id'], how='left')
print('After grant year school join')
print(df_e.shape)

# join ed champion
df_e = df_e.merge(df_champ, on='state', how='left')
print('After ed champ map')
print(df_e.shape)

# join district grant flag
df_e = df_e.merge(grants_district, on=['state', 'district_id'], how='left')
print('After district grant flag')
print(df_e.shape)

# join school grant flag
df_e = df_e.merge(grants_school, on=['state', 'district_id', 'school_id'], how='left')
print('After school grant flag')
print(df_e.shape)

# join grant details
df_e = df_e.merge(df_grantdetails_d, on=['state', 'district_id'], how='left')
print('After district grant details')
print(df_e.shape)
df_e = df_e.merge(df_grantdetails_s, on=['state', 'district_id', 'school_id'], how='left')
print('After school grant details')
print(df_e.shape)

print(df_e['state'].drop_duplicates())
print(df_e.head(5))
print(df_e.shape)

Importing ca
Pre city map
(2235695, 9)
Post city map
(2235695, 11)
Importing dc
Pre city map
(21606, 9)
Post city map
(21606, 11)
Importing ga
Pre city map
(111510, 9)
Post city map
(111510, 11)
Importing la
Pre city map
(214783, 9)
Post city map
(214783, 11)
Importing ma
Pre city map
(238368, 9)
Post city map
(238368, 11)
Importing mn
Pre city map
(826404, 9)
Post city map
(826404, 11)
Importing mo
Pre city map
(130604, 9)
Post city map
(130604, 11)
Importing nj
Pre city map
(294300, 9)
Post city map
(294300, 11)
Importing pa
Pre city map
(255283, 9)
Post city map
(255283, 11)
Importing tx
Pre city map
(1477200, 9)
Post city map
(1477200, 11)
After all appends
(5805753, 11)
After grade/group map
(2795138, 11)
After grant year join
(2795138, 12)
After grant year district join
(2795138, 13)
After grant year school join
(2795138, 14)
After ed champ map
(2795138, 21)
After district grant flag
(2795138, 22)
After school grant flag
(2795138, 23)
After district grant details
(2795138, 26)
Af

In [94]:
# import proficiency files
df_p = pd.DataFrame()
df_unjoined_cities = pd.DataFrame()

for state in states:
    print('Processing ' + state)
    try:
        temp = pd.read_csv(finalized_path + state + '_proficiency.csv',dtype={'year': str,'district_id': str,'district': str,'school_id': str,'school': str,'grade': str,'group_state': str,'subject': str,'proficient_tf': bool,'num_at_level': np.float64,'num_tested': np.float64,'pct_at_level': np.float64})
    except:
        print('No proficiency file for ' + state)
        continue
    # apply cleanStrings function to district_id and school_id
    temp['district_id'] = temp.apply(lambda x: cleanStrings(x, 'district_id') ,axis=1)
    temp['school_id'] = temp.apply(lambda x: cleanStrings(x, 'school_id') ,axis=1)
    temp['state'] = state

    # merge subject_map, group_map, and grade_map
    print(temp.shape)
    temp = temp.merge(df_grademap, on=['state','grade'], how='left')
    temp[temp.grade_new.isnull()][['state', 'grade']].drop_duplicates().to_csv('unmatched_grade_proficiency.csv')
    temp = temp[~temp.grade_new.isnull()]
    print('After grade lookups')
    print(temp.shape)
    temp = temp.merge(df_groupmap, on=['state','group_state'], how='left')
    temp[temp.group_new.isnull()][['state', 'group_state']].drop_duplicates().to_csv('unmatched_group_proficiency.csv')
    temp = temp[~temp.group_new.isnull()]
    print('After group lookups')
    print(temp.shape)
    temp = temp.merge(df_subjectmap, on=['state','subject'], how='left')
    temp[temp.subject_new.isnull()][['state', 'subject']].drop_duplicates().to_csv('unmatched_subject_proficiency.csv')
    temp = temp[~temp.subject_new.isnull()]
    print('After subject lookups')
    print(temp.shape)
    
    # roll up results by averaged pct_at_level
    temp = temp.groupby(['year', 'state', 'district_id', 'district', 'school_id', 'school', 'grade_new', 'group_new', 'subject_new'], as_index=False)['pct_at_level'].mean()
    temp = temp.rename(columns={'grade_new': 'grade', 'group_new': 'group', 'subject_new':'subject'})
    print('After roll up')
    print(temp.shape)
    
    # create composite subject by averaging pct_at_level as a simple average of cleaned ELA/Math subjects
    temp_comp = temp[temp.subject.isin(['ELA','Math'])]
    temp_comp = temp_comp.groupby(['year','state', 'district_id', 'district', 'school_id', 'school', 'grade', 'group'], as_index=False)['pct_at_level'].mean()
    temp_comp['subject'] = 'Composite'
    temp = temp.append(temp_comp, ignore_index = True)
    del temp_comp
    print('after composite')
    print(temp.shape)
    
    # join city file
    if state == 'tx':
        temp_dir = df_directory[['state', 'district_id', 'in_city', 'charter', 'flag']].drop_duplicates()
        temp = temp.merge(temp_dir, on=['state', 'district_id'], how = 'left')
        del temp_dir
    else:
        temp = temp.merge(df_directory, on=['state', 'district_id', 'school_id'], how = 'left')
        temp['charter'] = temp.apply(lambda x: 1 if x['state'] == 'dc' and x['district_id'].strip() != '1' else x['charter'], axis = 1)
        temp['in_city'] = temp.apply(lambda x: 1 if x['state'] == 'dc' else x['in_city'], axis = 1)
    # append unjoined records to df_unjoined_cities for Steve
    temp[temp.flag != 1].to_csv('unjoined_city_flag.csv')
    del temp['flag']
    temp['charter'] = temp['charter'].fillna('0')
    temp['in_city'] = temp['in_city'].fillna('0')
    print('after city file')
    print(temp.shape)
    # create state average & state std dev
    temp_avg = temp.groupby(['year', 'grade', 'subject', 'group'], as_index = False)['pct_at_level'].mean()
    temp_dev = temp.groupby(['year', 'grade', 'subject', 'group'])['pct_at_level'].std()
    # merge state average & std dev
    temp = temp.merge(temp_avg, on=['year', 'grade', 'subject', 'group'])
    temp = temp.merge(temp_dev, on=['year', 'grade', 'subject', 'group'])
    temp = temp.rename(columns={'pct_at_level': 'st_std', 'pct_at_level_y': 'st_avg', 'pct_at_level_x': 'pct_at_level'})
    del temp_avg
    del temp_dev
    # create state z score
    temp['st_z'] = (temp['pct_at_level'] - temp['st_avg']) / temp['st_std']
    temp['above_st_avg'] = temp['pct_at_level'] > temp['st_avg']
    print('after state z')
    print(temp.shape)

    # create city average
    temp_avg = temp.groupby(['year', 'grade', 'subject', 'group', 'in_city'], as_index = False)['pct_at_level'].mean()
    # create city std dev
    temp_dev = temp.groupby(['year', 'grade', 'subject', 'group', 'in_city'])['pct_at_level'].std()
    # merge city average & std dev
    temp = temp.merge(temp_avg, on=['year', 'grade', 'subject', 'group', 'in_city'])
    temp = temp.merge(temp_dev, on=['year', 'grade', 'subject', 'group', 'in_city'])
    temp = temp.rename(columns={'pct_at_level': 'city_std', 'pct_at_level_y': 'city_avg', 'pct_at_level_x': 'pct_at_level'})
    del temp_avg
    del temp_dev
    # create city z score
    temp['city_z'] = (temp['pct_at_level'] - temp['city_avg']) / temp['city_std']
    temp['above_city_avg'] = temp['pct_at_level'] > temp['city_avg']
    print('after city z')
    print(temp.shape)

    # merge all students result to LI student result
    temp_all = temp[temp.group.eq('All Students')]
    temp_all = temp_all.rename(columns={'st_avg': 'st_avg_all', 'city_avg': 'city_avg_all'})
    temp_all = temp_all[temp_all.grade.eq('All')]
    temp_all = temp_all[temp_all.subject.eq('Composite')]
    temp_li = temp[temp.group.eq('Low-Income')]
    temp_li = temp_li.rename(columns={'st_avg': 'st_avg_li','city_avg': 'city_avg_li'})
    temp_li = temp_li[temp_li.grade.eq('All')]
    temp_li = temp_li[temp_li.subject.eq('Composite')]
    temp_gap = temp_all.merge(temp_li, on=['year', 'district_id', 'district', 'school_id', 'school', 'subject', 'grade'], how='inner')
    del temp_li
    del temp_all
    # compare school's LI avg to the statewide all students avg as a 1/0 field (>)
    temp_gap['st_gap_close'] = temp_gap['st_avg_all'] < temp_gap['pct_at_level_y']
    # compare school's LI avg to the citywide all students avg as a 1/0 field (>)
    temp_gap['city_gap_close'] = temp_gap['city_avg_all'] < temp_gap['pct_at_level_y']

    # join gap close back to main dataframe
#     temp_gap_st = temp_gap[temp_gap['st_gap_close']]
#     temp_gap_st = temp_gap_st[['year', 'district_id', 'school_id', 'st_gap_close']].drop_duplicates()
#     temp_gap_city = temp_gap[temp_gap['city_gap_close']]
    temp_gap = temp_gap[['year','district_id', 'school_id', 'st_gap_close', 'city_gap_close']].drop_duplicates()
    temp = temp.merge(temp_gap, on=['year', 'district_id', 'school_id'], how='left')
#     temp = temp.merge(temp_gap, on=['year', 'district_id', 'school_id'], how='left')
    print('After gap closing')
    print(temp.shape)
#     del temp_gap_st
#     del temp_gap_city

    # create separate dataframe for previous year's results and join to year+1 (e.g. 2018 will get 2017's z joined)
    temp_prev = temp.copy()
    temp_prev['prev_year'] = temp_prev['year'].astype('int64') + 1
    temp_prev['prev_year'] = temp_prev['prev_year'].astype(str)
    temp_prev = temp_prev[['prev_year', 'district_id', 'school_id', 'grade', 'group', 'subject', 'st_z', 'city_z']]
    temp_prev = temp_prev.rename(columns={'prev_year': 'year'})
    temp = temp.merge(temp_prev, on=['year', 'district_id', 'school_id', 'grade', 'group', 'subject'], suffixes=('', '_prev'), how='left')
    del temp_prev
    # calculate improvement using an offset to the previous row (current z score - previous z score)
    temp['st_imp'] = temp['st_z'] - temp['st_z_prev']
    temp['city_imp'] = temp['city_z'] - temp['city_z_prev']

    # create improvement state average & state std dev
    temp_avg = temp.groupby(['year', 'grade', 'subject', 'group'], as_index = False)['st_imp'].mean()
    temp_dev = temp.groupby(['year', 'grade', 'subject', 'group'])['st_imp'].std()
    # merge improvment state average & std dev
    temp = temp.merge(temp_avg, on=['year', 'grade', 'subject', 'group'])
    temp = temp.merge(temp_dev, on=['year', 'grade', 'subject', 'group'])
    temp = temp.rename(columns={'st_imp': 'st_std_imp', 'st_imp_y': 'st_avg_imp', 'st_imp_x': 'st_imp'})
    del temp_avg
    del temp_dev
    # create state z score
    temp['st_z_imp'] = (temp['st_imp'] - temp['st_avg_imp']) / temp['st_std_imp']

    # create city average
    temp_avg = temp.groupby(['year', 'grade', 'subject', 'group', 'in_city'], as_index = False)['city_imp'].mean()
    # create city std dev
    temp_dev = temp.groupby(['year', 'grade', 'subject', 'group', 'in_city'])['city_imp'].std()
    # merge city average & std dev
    temp = temp.merge(temp_avg, on=['year', 'grade', 'subject', 'group', 'in_city'])
    temp = temp.merge(temp_dev, on=['year', 'grade', 'subject', 'group', 'in_city'])
    temp = temp.rename(columns={'city_imp': 'city_std_imp', 'city_imp_y': 'city_avg_imp', 'city_imp_x': 'city_imp'})
    del temp_avg
    del temp_dev
    # create city z score
    temp['city_z_imp'] = (temp['city_imp'] - temp['city_avg_imp']) / temp['city_std_imp']

    # create lists of distinct grade, group_state, subject, year
    years = temp.year.unique()
    grades = temp.grade.unique()
    subjects = temp.subject.unique()
    groups = temp.group.unique()
    # create quintiles by looping through year/grade/subject/group
    temp2 = pd.DataFrame()
    for year in years:
        for grade in grades:
            for subject in subjects:
                for group in groups:
                    temp3 = temp.loc[(temp['year'] == year) & (temp['grade'] == grade) & (temp['subject'] == subject) & (temp['group'] == group)]
                    # create state quintile
                    temp3['st_quin'] = pd.qcut(temp3['st_z'], 5, labels=False, duplicates='drop')
                    temp3['st_quin'] = temp3['st_quin'] + 1
                    temp3['top_20_st'] = temp3['st_quin'] == 5
                    # create city quintile
                    temp3['city_quin'] = pd.qcut(temp3['city_z'], 5, labels=False, duplicates='drop')
                    temp3['city_quin'] = temp3['city_quin'] + 1
                    temp3['top_20_city'] = temp3['city_quin'] == 5
                    # create improvement state quintile
                    temp3['st_quin_imp'] = pd.qcut(temp3['st_z_imp'], 5, labels=False, duplicates='drop')
                    temp3['st_quin_imp'] = temp3['st_quin_imp'] + 1
                    # create improvement city quintile
                    temp3['city_quin_imp'] = pd.qcut(temp3['city_z_imp'], 5, labels=False, duplicates='drop')
                    temp3['city_quin_imp'] = temp3['city_quin_imp'] + 1
                    temp2 = temp2.append(temp3, ignore_index=True)

    df_p = df_p.append(temp2, ignore_index = True, sort = True)
    
print(df_p['state'].drop_duplicates())

Processing ca
(709291, 14)
After grade lookups
(709291, 16)
After group lookups
(709291, 18)
After subject lookups
(709291, 20)
After roll up
(709291, 10)
after composite
(1069045, 10)
after city file
(1069045, 12)
after state z
(1069045, 16)
after city z
(1069045, 20)
After gap closing
(1069045, 22)


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:177: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:178: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

Processing dc
(141645, 14)
After grade lookups
(141645, 16)
After group lookups
(64120, 18)
After subject lookups
(64120, 20)
After roll up
(28102, 10)
after composite
(42321, 10)
after city file
(42321, 12)
after state z
(42321, 16)
after city z
(42321, 20)
After gap closing
(42321, 22)
Processing ga
(2457838, 13)
After grade lookups
(2457838, 15)
After group lookups
(1429539, 17)
After subject lookups
(1429539, 19)
After roll up
(685548, 10)
after composite
(885809, 10)
after city file
(885809, 12)
after state z
(885809, 16)
after city z
(885809, 20)
After gap closing
(885809, 22)
Processing la
(616394, 11)
After grade lookups
(616394, 13)
After group lookups
(269699, 15)
After subject lookups
(269699, 17)
After roll up
(195532, 10)
after composite
(288257, 10)
after city file
(288257, 12)
after state z
(288257, 16)
after city z
(288257, 20)
After gap closing
(288257, 22)
Processing ma
(685406, 13)
After grade lookups
(685406, 15)
After group lookups
(329899, 17)
After subject lookup

In [95]:
# join grants on state & district_id, and by state, district_id, school_id
# df_p = df_p.merge(grants_district, on=['state', 'district_id'], suffixes=('', '_grant_district'), how='outer')
# df_p = df_p.merge(grants_school, on=['state', 'district_id', 'school_id'], suffixes=('', '_grant_school'), how='outer')

# join grant year
df_p = df_p.merge(df_grantyear, on=['state'], how='left')
print('After grant year join')
print(df_p.shape)

# join grant year district
df_p = df_p.merge(df_grantyear_d, on=['state', 'district_id'], how='left')
print('After grant year district join')
print(df_p.shape)

# join grant year school
df_p = df_p.merge(df_grantyear_s, on=['state', 'district_id', 'school_id'], how='left')
print('After grant year school join')
print(df_p.shape)

# join ed champion
df_p = df_p.merge(df_champ, on='state', how='left')

# join district grant flag
df_p = df_p.merge(grants_district, on=['state', 'district_id'], how='left')
print('After district grant flag')
print(df_p.shape)

# join school grant flag
df_p = df_p.merge(grants_school, on=['state', 'district_id', 'school_id'], how='left')
print('After school grant flag')
print(df_p.shape)

# join grant details
df_p = df_p.merge(df_grantdetails_d, on=['state', 'district_id'], how='left')
print('After district grant details')
print(df_p.shape)
df_p = df_p.merge(df_grantdetails_s, on=['state', 'district_id', 'school_id'], how='left')
print('After school grant details')
print(df_p.shape)

After grant year join
(7216551, 39)
After grant year district join
(7216551, 40)
After grant year school join
(7216551, 41)
After district grant flag
(7216551, 49)
After school grant flag
(7216551, 50)
After district grant details
(7216551, 53)
After school grant details
(7216551, 56)


In [96]:
# MN - schoolid 50 districtId 177 Wyndam Senior High
# CA - schoolid 118380 districtid 69484 Christopher High
# LA - schoolid 17102 districtid 17 Woodlawn High School
# TX - schoolid 200901001 districtid 200901 Balanger High School
# PA - schoolid 6793 districtid 103026852 North Allegheny

# add grade config
# create dataframe of distinct state/district/school/grade from enrollment data
df_grades = df_e[['state', 'district_id', 'school_id', 'grade']].append(df_p[['state', 'district_id', 'school_id', 'grade']], ignore_index=True, sort=True).drop_duplicates()

# df_grades = df_grades[df_grades.state.eq('mn') & df_grades.school_id.eq('50') & df_grades.district_id.eq('177')]

#drop all grades records
df_grades = df_grades[~df_grades.grade.eq('All')]

# if grade = HS/EOC, then grade = '9'
df_grades['grade'] = [x if x != 'HS/EOC' else '9' for x in df_grades['grade']]

df_grades = df_grades.drop_duplicates()

# convert grade to number
def toFloat(row, column):
    try:
        return float(row[column])
    except:
        return np.nan
df_grades['grade'] = df_grades.apply(lambda x: toFloat(x, 'grade'), axis=1)
print(df_grades.shape)

df_grades = df_grades.dropna(subset=['grade'])
print(df_grades.shape)
# calculate max grade
df_max = df_grades.groupby(['state', 'district_id', 'school_id'], as_index=False)['grade'].max()
df_max = df_max.rename(columns={'grade':'max'})
# calculate min grade
df_min = df_grades.groupby(['state', 'district_id', 'school_id'], as_index=False)['grade'].min()
df_min = df_min.rename(columns={'grade':'min'})
# calculate grade5? and grade8?
df_5 = df_grades[df_grades.grade.eq(5)]
df_5['grade5flag'] = 1
del df_5['grade']
df_8 = df_grades[df_grades.grade.eq(8)]
df_8['grade8flag'] = 1
del df_8['grade']

# merge all together
df_grades = df_max.merge(df_min, on=['state', 'district_id', 'school_id'])
df_grades = df_grades.merge(df_5, on=['state', 'district_id', 'school_id'], how='left')
df_grades = df_grades.merge(df_8, on=['state', 'district_id', 'school_id'], how='left')
del df_max
del df_min
del df_5
del df_8

def gradeConfig(row):
    if row['grade5flag'] == 1 and row['grade8flag'] == 1:
        if row['max'] <= 8:
            return 'ESMS'
        else:
            return 'ESMSHS'
    if row['min'] >= 5:
        if row['max'] <= 8:
            return 'MS'
        elif row['min'] >= 9:
            return 'HS'
        else:
            return 'MSHS'
    if row['max'] <= 7:
        return 'ES'
    if row['state'] == 'nj' and row['school_id'] == '300' and row['district_id'] == '1431':
        return 'ESMS'

df_grades['grade_config'] = df_grades.apply(lambda x: gradeConfig(x), axis = 1)
print(df_grades.head(5))
df_grades[df_grades['grade_config'].isnull()].to_csv('unmatched_grade_config.csv')
# ES = if highest grade is <=7
# ESMS = if has both grade 5 and 8 and no grade >8
# MS = if lowest grade is >=5 and highest grade <=8
# MSHS = if lowest grade >=5 and highest grade >=9
# ESMSHS = if has grade 5, grade 8 and any grade >=9
# HS = if lowest grade >=9 and highest grade <=12
print(df_grades.groupby(['grade_config'])['state'].count())

(163531, 4)
(163531, 4)


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


  state district_id school_id  max  min  grade5flag  grade8flag grade_config
0    ca       10017    109835  9.0  1.0         1.0         1.0       ESMSHS
1    ca       10017    112607  9.0  9.0         NaN         NaN           HS
2    ca       10017    118489  9.0  9.0         NaN         NaN           HS
3    ca       10017    123968  8.0  1.0         1.0         1.0         ESMS
4    ca       10017    124172  8.0  1.0         1.0         1.0         ESMS
grade_config
ES        17870
ESMS       2915
ESMSHS     2925
HS         6426
MS         3162
MSHS       4660
Name: state, dtype: int64


In [97]:
# merge grade config back to enrollment data
print(df_e.shape)
df_e = df_e.merge(df_grades, on=['state', 'district_id', 'school_id'], how='left')
df_e['grade_config'] = df_e['grade_config'].fillna('Unknown')
print(df_e.shape)
print(df_p.shape)
df_p = df_p.merge(df_grades, on=['state', 'district_id', 'school_id'], how='left')
df_p['grade_config'] = df_p['grade_config'].fillna('Unknown')
print(df_p.shape)
del df_grades

(2795138, 29)
(2795138, 34)
(7216551, 56)
(7216551, 61)


In [98]:
# join enrollment numbers on year, district_id, school_id, grade, group, maintaining records from both
# df = df_p.merge(df_e, on=['year', 'state', 'district_id', 'school_id', 'grade', 'group'], how='outer', suffixes=('_p', '_e'))

# print(df.shape)
# print(df['state'].drop_duplicates())

In [99]:
# join proficiency metrics needed in dashboards to enrollment file
df_scores = df_p[df_p.subject.eq('Composite')]
df_scores = df_scores[['year', 'state', 'district_id', 'school_id', 'grade', 'group', 'above_st_avg', 'above_city_avg', 'top_20_st', 'top_20_city', 'city_gap_close', 'st_gap_close', 'city_quin', 'st_quin']].drop_duplicates()
print(df_e.shape)
df_e = df_e.merge(df_scores, on=['year', 'state', 'district_id', 'school_id', 'grade', 'group'], how='left')
print(df_e.shape)

(2795138, 34)
(2796620, 42)


In [100]:
# qa gap closing
temp = df_e[df_e.st_gap_close.eq(True) | df_e.city_gap_close.eq(True)]
print(temp.groupby(['state']).school_id.nunique())

state
ca    3949
dc     101
ga     250
la     657
ma     120
mn     251
mo     176
nj      89
tx    5511
Name: school_id, dtype: int64


In [101]:
# export enrollment and proficiency dataframes
# df_e.to_csv('data/finalized/enrollment.csv')
# df_p.to_csv('data/finalized/proficiency.csv')
df_e.to_csv('data/finalized/enrollment_finalized.csv')
df_p.to_csv('data/finalized/proficiency_finalized.csv')

In [102]:
# export grants data
df_grant.to_csv('data/finalized/grants_finalized.csv')

In [103]:
# df_p.groupby(['state', 'year', 'grade', 'group'])['school_id'].count().to_csv('temp.csv')
# print(df_p.groupby(['year'])['school_id'].nunique())

In [104]:
# export df
# df.to_csv('data/finalized/final.csv')